In [1]:
import tweepy
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import time
import math

In [2]:
CONSUMER_KEY = 'vc78cacd7TdgCWEWQqJ2wdPQf'
CONSUMER_SECRET = '8qS0VyWY7CVKgyUbfhcdIM0PuixzzRvLbpW17QlJNZv8Q3i7hh'
ACCESS_TOKEN = '80123742-yOIsvgbrG2qbpGMlAREVNoE7ICZHf51dI2EGAzgH1'
ACCESS_TOKEN_SECRET = 'msLbM8YGYlwAFUZIkMkVfZqrP4Am1AXDqcEGs067sF4hG'

In [3]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
MONGO_HOST = 'mongodb://localhost:27017/'

client = MongoClient(MONGO_HOST)

db = client['IWD2021']

collection = db.tweets

collection2 = db.tweets_server_updated

In [6]:
num_docs = collection.count_documents({})
num_limit = 1000
num_skip = 0
num_docs

5631905

In [7]:
num_page = 0
pages = math.ceil(num_docs/num_limit)
pages

5632

In [8]:
print('Atualização de tweets iniciada')
print(f'{time.asctime(time.localtime(time.time()))}')

while num_page < pages:
    print(f'\nBloco {num_page + 1}/{pages}')

    docs = []
    tweets_ids = []

    docs = list(collection.find({}, {'id': True}
                                ).limit(num_limit).skip(num_skip))
    tweets_ids = [doc['id'] for doc in docs]

    print(f'Extraindo {len(tweets_ids)} tweets para atualização\n')

    data_to_update = {}
    data = []
    key = 0

    for tweet_id in tweets_ids:
        data.append(tweet_id)
        data_to_update[key] = data
        if(len(data) == 100):
            key = key + 1
            data = []

    for d in data_to_update.keys():
        updated_tweets = []
        tweets = []

        updated_tweets = api.statuses_lookup(
            data_to_update[d], tweet_mode='extended')
        tweets = [ut._json for ut in updated_tweets]
        collection2.insert_many(tweets)

        num_skip = num_skip + len(data_to_update[d])

        print(
            f'{len(data_to_update[d])} tweets enviados | {len(tweets)} atualizados | Total enviado: {num_skip}/{num_docs} | {(num_skip*100)/num_docs:.2f}%')

    num_page = num_page + 1

print('Atualização de tweets finalizada')
print(time.asctime(time.localtime(time.time())))

Atualização de tweets iniciada
Wed Mar 24 00:16:17 2021

Bloco 1/5632
Extraindo 1000 tweets para atualização

100 tweets enviados | 92 atualizados | Total enviado: 100/5631905 | 0.00%
100 tweets enviados | 92 atualizados | Total enviado: 200/5631905 | 0.00%
100 tweets enviados | 95 atualizados | Total enviado: 300/5631905 | 0.01%
100 tweets enviados | 96 atualizados | Total enviado: 400/5631905 | 0.01%
100 tweets enviados | 93 atualizados | Total enviado: 500/5631905 | 0.01%
100 tweets enviados | 93 atualizados | Total enviado: 600/5631905 | 0.01%
100 tweets enviados | 89 atualizados | Total enviado: 700/5631905 | 0.01%
100 tweets enviados | 93 atualizados | Total enviado: 800/5631905 | 0.01%
100 tweets enviados | 98 atualizados | Total enviado: 900/5631905 | 0.02%
100 tweets enviados | 95 atualizados | Total enviado: 1000/5631905 | 0.02%

Bloco 2/5632
Extraindo 1000 tweets para atualização

100 tweets enviados | 92 atualizados | Total enviado: 1100/5631905 | 0.02%
100 tweets enviados 

KeyboardInterrupt: 

In [ ]:
# collection2.drop()